### Setup

In [1]:
import pandas as pd
import numpy as np
import nltk.corpus
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Import data

In [2]:
chatgpt_prompt2 = pd.read_csv("https://raw.githubusercontent.com/lodi-m/u-integrity/main/data/chatgpt_prompt2_essays.csv")

#taking last half of data
student_data = pd.read_excel("/content/drive/MyDrive/U-Integrity BorealisAI ML Program/data/asap_aes_data.xlsx")
student_data = student_data[student_data['essay_set'] == 2].tail(900)

In [11]:
chatgpt_prompt2.head(5)

,Unnamed: 0,gen_essay
0,0,opinion librari censor materi fact believ fund...
1,1,censorship alway controversi topic propon oppo...
2,2,univers answer question sinc everyon differ op...
3,3,person believ book materi allow access everyon...
4,4,censorship librari hot topic day one side fenc...


In [12]:
student_data.head(5)

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,...,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6
2683,3878,2,certain kind book librari book tooken shelf pe...,3.0,4.0,NaN,3.0,3.0,4.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2684,3879,2,think censorship stupid come put adult show se...,1.0,1.0,NaN,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2685,3880,2,dear newspap censorship librari peopl certain ...,3.0,3.0,NaN,3.0,4.0,4.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2686,3881,2,take moment go back highschool rememb roam ope...,5.0,5.0,NaN,5.0,4.0,4.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2687,3882,2,bunch good book sat public libari get kind dif...,3.0,2.0,NaN,3.0,3.0,2.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Data Cleaning Functions

In [5]:
def normalize_text(df, col):
  df[col] = df[col].str.lower()
  return df[col]

In [6]:
def remove_punctuation(df, col):
  df[col] = df[col].str.replace(r'(?<!\w)@\w+', '') #tags
  df[col] = df[col].str.replace(r'(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?', '') #punctuation
  df[col] = df[col].replace(r'\n',' ', regex=True) #newlines
  return df[col]

In [7]:
def remove_stopwords_stem(df, col):
  stop = list(stopwords.words('english'))
  stemmer = PorterStemmer()

  df[col] = df[col].apply(lambda x: [word for word in x.split() if word not in stop])
  df[col] = df[col].apply(lambda x: [stemmer.stem(word) for word in x])
  df[col] = df[col].apply(' '.join)

  return df[col]

In [8]:
def text_preprocessing(df, col):
  normalize_text(df, col)
  remove_punctuation(df, col)
  remove_stopwords_stem(df, col)

  return df

### Saving final dfs as csvs

In [9]:
final_chatgpt = text_preprocessing(chatgpt_prompt2, 'gen_essay')['gen_essay']
final_student = text_preprocessing(student_data, 'essay')['essay']

<ipython-input-6-09f12dd6abef>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df[col] = df[col].str.replace(r'(?<!\w)@\w+', '') #tags
<ipython-input-6-09f12dd6abef>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df[col] = df[col].str.replace(r'(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?', '') #punctuation


In [10]:
final_chatgpt.to_csv('chatgpt_essay_set_2.csv', index=False)
final_student.to_csv('kaggle_essay_set_2.2.csv', index=False)